# Houston 311 Service Requests Data Mining Project

## Group members
Stephen Huang

Levi Villarreal

Joshua Wong

Andrew Young


### Code Imports

In [1]:
import pandas as pd

### Import data

In [6]:
# Data is from http://www.houstontx.gov/311/
names = ["CASE NUMBER", "SR LOCATION", "COUNTY", "DISTRICT", "NEIGHBORHOOD", "TAX ID", 
         "TRASH QUAD", "RECYCLE QUAD", "TRASH DAY", "HEAVY TRASH DAY", "RECYCLE DAY", 
         "KEY MAP", "MANAGEMENT DISTRICT", "DEPARTMENT", "DIVISION", "SR TYPE", "QUEUE", 
         "SLA", "STATUS", "SR CREATE DATE", "DUE DATE", "DATE CLOSED", "OVERDUE", "Title", 
         "x", "y", "LATITUDE", "LONGITUDE", "Channel Type"]

data = pd.read_csv('311-Public-Data-Extract-2018-clean.txt', sep="|", header=None, names=names)
data.head

<bound method NDFrame.head of                   CASE NUMBER  \
0                 CASE NUMBER   
1                101002866096   
2       12202635-101002866097   
3       12202636-101002866098   
4       12202637-101002866099   
...                       ...   
399948           101003318948   
399949           101003318949   
399950           101003318950   
399951           101003318951   
399952           101003318952   

                                              SR LOCATION         COUNTY  \
0                                             SR LOCATION         COUNTY   
1       Intersection 13300 HEMPSTEAD RD&9100 PINEMONT ...  Harris County   
2                       2803 W BELLFORT, HOUSTON TX 77054            NaN   
3                           2115 HUTTON, HOUSTON TX 77026         HARRIS   
4                   14806 STEEPLE CHASE, HOUSTON TX 77489      FORT BEND   
...                                                   ...            ...   
399948                      1412 ASBURY, 